# Convolutional Neural Network for Digit Recognition

This notebook trains a Convolutional Neural Network (CNN) for digit (i.e. 0, 1, ..., 9) recognition using the MNIST dataset.

In [ ]:
import os
import sys
import zipfile
import random
import math
import re
import time
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

sys.path.insert(0, '../libraries')
import cnn.cnn
import cnn.config

%matplotlib inline
%config IPCompleter.greedy=True

HOME_DIR = '/home/keras'
ROOT_DATA_DIR = os.path.join(HOME_DIR, "data")
WEIGHTS_DIR = os.path.join(ROOT_DATA_DIR, "weights")
MODEL_DIR = os.path.join(ROOT_DATA_DIR, "logs")

# Configuration

Configure the model's hyperparameters and training settings.

In [ ]:
class MnistConfig(cnn.config.Config):
    NAME = "mnist_cnn"
    EPOCHS = 1
    EPOCH_STEPS = 100
    CLASSES = 10  # 10 digits
    IMAGE_HEIGHT = 28
    IMAGE_WIDTH = 28
    
config = MnistConfig()
config.display()

# Dataset

Load the dataset. Download if required, and transform into required format.

In [ ]:
import keras.backend as KB
import keras.utils as KU
from keras.datasets import mnist

(x_train, y_train), (x_validate, y_validate) = mnist.load_data()

if KB.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, config.IMAGE_HEIGHT, config.IMAGE_WIDTH)
    x_validate = x_validate.reshape(x_validate.shape[0], 1, config.IMAGE_HEIGHT, config.IMAGE_WIDTH)
else:
    x_train = x_train.reshape(x_train.shape[0], config.IMAGE_HEIGHT, config.IMAGE_WIDTH, 1)
    x_validate = x_validate.reshape(x_validate.shape[0], config.IMAGE_HEIGHT, config.IMAGE_WIDTH, 1)

x_train = x_train.astype('float32')
x_validate = x_validate.astype('float32')
x_train /= 255
x_validate /= 255

# convert class vectors to binary class matrices
y_train = KU.to_categorical(y_train, config.CLASSES)
y_validate = KU.to_categorical(y_validate, config.CLASSES)

dataset_train = {'inputs': x_train, 'labels': y_train}
dataset_validate = {'inputs': x_validate, 'labels': y_validate}

# Model

Create a model instance. This is also where you would load any pretrained weights.

In [ ]:
model = cnn.cnn.CNN(config=config, model_dir=MODEL_DIR)

# Train

Train the model. If using pretrained weights, first train the head of the model before fine tuning lower sections.

In [ ]:
model.train(dataset_train, dataset_validate)

# Test

Test how the model performs on sample input data. Load weights to test without training during this session. You still have to train at some point though.

In [ ]:
weights = cnn.cnn.find_last(config, MODEL_DIR)[1]
model.keras_model.load_weights(weights)

In [ ]:
test_index = random.randint(1,10000)
test_input = dataset_validate.get('inputs')[test_index, :, :, 0]
test_label = dataset_validate.get('labels')[test_index]
pil_input = Image.fromarray(np.uint8(test_input*255))
plt.imshow(pil_input)

prediction = model.predict(pil_input)

print("label: {}".format(np.argmax(test_label)))
print("prediction: {}".format(prediction))


# Evaluate

Evaluate the performance of the model on the test dataset. 

In [ ]:
model.evaluate(dataset_validate)